## process:0 Sidelink merge
#### step:1 working with pcap data
dfs = []
for i in range(1, 5):
    df = pd.read_csv(f"ue{i}_....")
    df['Destination'] = i
    dfs.append(df)
df = pd.concat(dfs)

##### step:1.1 rename/mapping/resampling
###### rename
df['Source'] = df['Source'].map({'abcd': 1, ...})
###### add packet count for PER
df['Received Packets'] = 1
###### resampling
```
df = df.groupby(['Source', 'Destination']).resample('1s').aggregate({'Received Packets': 'sum', 'RSRP': 'mean', ... [add all other columns]})
df.reset_index(inplace=True)
```

#### step:2 working with gps data
```

dfs = []
for i in range(1, 5):
    df_gps = pd.read_csv(f"ue{i}_....")
    df_gps['Device'] = i
    dfs.append(df_gps)
df_gps = pd.concat(dfs)

```



#### step:3 merge with gps data with pcap
```
df_merged = pd.merge(df, df_gps, left_on=['timestamp', 'Source'], right_on=['timestamp', 'Device'], suffixes=(None, ' Source'))

df_merged = pd.merge(df_merged, df_gps, left_on=['timestamp', 'Destination'], right_on=['timestamp', 'Device'], suffixes=(None, ' Destination'))

```

## import

In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import os
import sys
import glob
import pandas as pd
from pathlib import Path

sys.path.append("../")

from utils import spherical_coords_to_planar_coords, geotag, write_meta

In [2]:
data_path = Path.cwd().parent/"data"
sidelink_raw = data_path/"sources/sidelink"
info_path = Path.cwd().parent/"metadata"
gps_path = data_path/"sources/gps"
period_seconds = 1
sample_period = f"{period_seconds}s"

## step:1 working with PCAP data

In [3]:
# concat all PCAP data here
df_pcap_list = []
for p in sidelink_raw.rglob("*.parquet"):
    df = pd.read_parquet(p).dropna()
    fname = p.stem
    UE = -1
    scenario = -1
    if 'ue1' in fname:
        UE = 1
    elif 'ue2' in fname:
        UE = 2
    elif 'ue3' in fname:
        UE = 3
    elif 'ue4' in fname:
        UE = 4
    else:
        raise ValueError(f"Unrecognized UE for {p}")
    if 's1' in fname:
        scenario = "S1"
    elif 's2' in fname:
        scenario = "S2"
    else:
        raise ValueError(f"Unrecognized scenario for {p}")
    df['Scenario'] = scenario
    df['Destination'] = UE
    df['Source'] = df['Source'].map({'53:4c:31':1,'53:4c:32': 2, '53:4c:33': 3, '53:4c:34': 4})
    df['Received Packets'] = 1
    df_pcap_list.append(df)

df_pcap = pd.concat(df_pcap_list)


In [4]:
df_pcap

,time_epoch,SNR,RSRP,RSSI,NOISE POWER,TIME(S),TIME(FrS),RX_GAIN,SubFrame_NUMBER,SubFrame_LENGHT,Rx_power,Source,Destination,MCS,Scenario,Received Packets
10596,1.624377e+09,16.9992,-70.7719,-48.1345,0.000904,1.624377e+09,0.283,45.0,748.0,3.0,0.049378,2.0,1,8,S1,1
10597,1.624377e+09,22.3486,-64.2144,-43.4314,0.001223,1.624377e+09,0.306,45.0,750.0,6.0,0.223502,3.0,1,8,S1,1
10598,1.624377e+09,16.7687,-72.0268,-51.1947,0.000721,1.624377e+09,0.322,45.0,752.0,2.0,0.036987,4.0,1,8,S1,1
10599,1.624377e+09,15.8171,-72.6514,-50.8650,0.000774,1.624377e+09,0.333,45.0,753.0,3.0,0.032032,2.0,1,8,S1,1
10600,1.624377e+09,21.4917,-64.8475,-43.6239,0.001286,1.624377e+09,0.356,45.0,755.0,6.0,0.193186,3.0,1,8,S1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872996,1.624461e+09,18.2991,-70.9774,-43.2089,0.000616,1.624461e+09,0.929,40.0,348.0,9.0,0.047096,2.0,4,12,S2,1
1872997,1.624461e+09,15.1978,-75.8824,-48.2197,0.000401,1.624461e+09,0.954,40.0,351.0,4.0,0.015222,2.0,4,12,S2,1
1872998,1.624461e+09,14.6238,-76.8553,-48.8240,0.000365,1.624461e+09,0.969,40.0,352.0,9.0,0.012167,2.0,4,12,S2,1
1872999,1.624461e+09,13.2598,-78.2930,-50.5637,0.000355,1.624461e+09,0.989,40.0,354.0,9.0,0.008738,2.0,4,12,S2,1


In [5]:
df_pcap['timestamp'] = pd.to_datetime(df_pcap['time_epoch'], utc=True, unit = 's')
df_pcap.timestamp = df_pcap.timestamp.dt.tz_convert('Europe/Berlin')
df_pcap.set_index("timestamp", inplace=True)
df_pcap.sort_index(inplace=True)

In [6]:
df_pcap

,time_epoch,SNR,RSRP,RSSI,NOISE POWER,TIME(S),TIME(FrS),RX_GAIN,SubFrame_NUMBER,SubFrame_LENGHT,Rx_power,Source,Destination,MCS,Scenario,Received Packets
timestamp,,,,,,,,,,,,,,,,
2021-06-22 09:40:28.255369984+02:00,1.624348e+09,5.23428,-82.1881,-58.3644,0.000794,1.624348e+09,0.254,31.0,585.0,4.0,0.003564,3.0,4,8,S1,1
2021-06-22 09:40:28.305282048+02:00,1.624348e+09,12.24780,-77.1943,-55.4423,0.000607,1.624348e+09,0.304,39.0,590.0,4.0,0.011254,3.0,4,8,S1,1
2021-06-22 09:40:28.355268864+02:00,1.624348e+09,16.69010,-74.7742,-53.9690,0.000395,1.624348e+09,0.354,39.0,595.0,4.0,0.019647,3.0,4,8,S1,1
2021-06-22 09:40:28.405411840+02:00,1.624348e+09,10.85600,-81.1737,-59.5069,0.000326,1.624348e+09,0.404,39.0,600.0,4.0,0.004501,3.0,4,8,S1,1
2021-06-22 09:40:28.464296960+02:00,1.624348e+09,15.90600,-69.0842,-47.8348,0.001750,1.624348e+09,0.463,47.0,606.0,3.0,0.072829,3.0,4,8,S1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 17:10:07.608491008+02:00,1.624461e+09,8.66732,-62.7936,-35.1746,0.034760,1.624461e+09,0.607,46.0,792.0,7.0,0.310003,1.0,3,12,S2,1
2021-06-23 17:10:07.638375168+02:00,1.624461e+09,8.25596,-59.7554,-32.1245,0.076330,1.624461e+09,0.637,46.0,795.0,7.0,0.623993,1.0,3,12,S2,1
2021-06-23 17:10:07.648569088+02:00,1.624461e+09,8.88441,-59.2677,-31.5538,0.075027,1.624461e+09,0.647,46.0,796.0,7.0,0.698151,1.0,3,12,S2,1


In [7]:
group_columns = ['Source', 'Destination', 'Scenario']
agg_method = {c: 'mean' for c in df_pcap.columns if c not in group_columns}
agg_method['Received Packets'] = 'sum'
agg_method['MCS'] = 'median'

df_resample = df_pcap.groupby(['Source', 'Destination', 'Scenario']).resample(sample_period).aggregate(agg_method)


In [8]:
df_resample.dropna(inplace=True)
df_resample

time_epoch  \
Source Destination Scenario timestamp                                 
1.0    1           S1       2021-06-23 10:46:06+02:00  1.624438e+09   
                            2021-06-23 10:46:07+02:00  1.624438e+09   
       2           S1       2021-06-22 09:43:54+02:00  1.624348e+09   
                            2021-06-22 09:43:55+02:00  1.624348e+09   
                            2021-06-22 09:43:56+02:00  1.624348e+09   
...                                                             ...   
4.0    4           S2       2021-06-23 14:43:05+02:00  1.624452e+09   
                            2021-06-23 14:59:10+02:00  1.624453e+09   
                            2021-06-23 15:26:13+02:00  1.624455e+09   
                            2021-06-23 15:37:19+02:00  1.624455e+09   
                            2021-06-23 16:31:17+02:00  1.624459e+09   

                                                             SNR       RSRP  \
Source Destination Scenario timestamp                                         
1.0    1           S1       2021-06-23 10:46:06+02:00  19.086750 -28.674486   
                            2021-06-23 10:46:07+02:00  18.887818 -40.548027   
       2           S1       2021-06-22 09:43:54+02:00  23.033555 -39.557575   
                            2021-06-22 09:43:55+02:00  23.950920 -41.161835   
                            2021-06-22 09:43:56+02:00  24.015625 -49.046845   
...                                                          ...        ...   
4.0    4           S2       2021-06-23 14:43:05+02:00  24.589943 -32.903986   
                            2021-06-23 14:59:10+02:00  24.563781 -33.329294   
                            2021-06-23 15:26:13+02:00  27.350300 -34.613800   
                            2021-06-23 15:37:19+02:00  23.778800 -33.943100   
                            2021-06-23 16:31:17+02:00  23.299200 -33.320000   

                                                            RSSI  NOISE POWER  \
Source Destination Scenario timestamp                                           
1.0    1           S1       2021-06-23 10:46:06+02:00  -7.871425     9.292362   
                            2021-06-23 10:46:07+02:00 -19.745309     0.789915   
       2           S1       2021-06-22 09:43:54+02:00 -18.657069     2.334353   
                            2021-06-22 09:43:55+02:00 -20.238961     0.732931   
                            2021-06-22 09:43:56+02:00 -28.222915     0.029224   
...                                                          ...          ...   
4.0    4           S2       2021-06-23 14:43:05+02:00  -5.119161     1.218360   
                            2021-06-23 14:59:10+02:00  -5.609721     1.059661   
                            2021-06-23 15:26:13+02:00  -6.832400     0.356641   
                            2021-06-23 15:37:19+02:00  -6.127300     0.945024   
                            2021-06-23 16:31:17+02:00  -5.591610     1.217460   

                                                            TIME(S)  \
Source Destination Scenario timestamp                                 
1.0    1           S1       2021-06-23 10:46:06+02:00  1.624438e+09   
                            2021-06-23 10:46:07+02:00  1.624438e+09   
       2           S1       2021-06-22 09:43:54+02:00  1.624348e+09   
                            2021-06-22 09:43:55+02:00  1.624348e+09   
                            2021-06-22 09:43:56+02:00  1.624348e+09   
...                                                             ...   
4.0    4           S2       2021-06-23 14:43:05+02:00  1.624452e+09   
                            2021-06-23 14:59:10+02:00  1.624453e+09   
                            2021-06-23 15:26:13+02:00  1.624455e+09   
                            2021-06-23 15:37:19+02:00  1.624455e+09   
                            2021-06-23 16:31:17+02:00  1.624459e+09   

                                                       TIME(FrS)    RX_GAIN  \
Source Destination Scenario timestamp           

In [9]:
df_temp = df_resample.dropna().reset_index().set_index('timestamp').sort_index()

In [10]:
df_temp[["Source", "MCS"]] = df_temp[["Source", "MCS"]].astype(int)

In [11]:
df_temp.to_parquet(data_path/"sidelink_intermediate.parquet", compression='gzip')

In [12]:
df_temp

,Source,Destination,Scenario,time_epoch,SNR,RSRP,RSSI,NOISE POWER,TIME(S),TIME(FrS),RX_GAIN,SubFrame_NUMBER,SubFrame_LENGHT,Rx_power,MCS,Received Packets
timestamp,,,,,,,,,,,,,,,,
2021-06-22 09:40:28+02:00,3,4,S1,1.624348e+09,15.680879,-61.244507,-39.893107,0.141945,1.624348e+09,0.605200,55.133333,620.133333,3.866667,11.784216,8,15
2021-06-22 09:40:29+02:00,3,4,S1,1.624348e+09,17.058347,-39.437879,-18.154800,1.346087,1.624348e+09,0.457579,76.000000,705.368421,3.894737,73.932863,8,19
2021-06-22 09:40:30+02:00,3,4,S1,1.624348e+09,17.290685,-39.063805,-17.859140,1.461924,1.624348e+09,0.480350,76.000000,807.650000,3.850000,86.270430,8,20
2021-06-22 09:40:31+02:00,3,4,S1,1.624348e+09,16.553859,-39.401645,-18.050220,1.595591,1.624348e+09,0.479000,76.000000,907.500000,4.000000,80.081815,8,20
2021-06-22 09:40:32+02:00,3,4,S1,1.624348e+09,18.107595,-39.480995,-18.242310,1.038822,1.624348e+09,0.484750,76.000000,649.550000,5.250000,76.324090,8,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 17:10:02+02:00,1,3,S2,1.624461e+09,6.205097,-63.793783,-36.070856,0.057210,1.624461e+09,0.249778,46.000000,256.777778,2.000000,0.273857,12,18
2021-06-23 17:10:03+02:00,1,3,S2,1.624461e+09,8.796810,-65.391500,-37.717800,0.018505,1.624461e+09,0.392000,46.000000,371.000000,2.000000,0.170440,12,1
2021-06-23 17:10:05+02:00,1,3,S2,1.624461e+09,11.293713,-66.601178,-38.845178,0.011680,1.624461e+09,0.913000,46.000000,622.666667,6.333333,0.140825,12,9


## Step: 2 
### working with GPS data

In [13]:
## Extract the GPS from the cellular data
gps_cols = ['ts_gps', 'Latitude', 'Longitude', 'Altitude',
            'speed_kmh', 'COG', 'precipIntensity',
            'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint',
            'humidity', 'pressure', 'windSpeed', 'cloudCover', 'uvIndex',
            'visibility', 'Traffic Jam Factor', 'Traffic Street Name',
            'Traffic Distance', 'Pos in Ref Round']

gps_list = []
for p in gps_path.rglob("*.parquet"):
    print(p.relative_to(gps_path))

    src = p.parent.stem
    pc = p.stem
         
    df = pd.read_parquet(p, columns=gps_cols)
    df["device"] = pc
    df["area"] = geotag(df.Latitude, df.Longitude)
    gps_list.append(df)
gps_df = pd.concat(gps_list).rename(columns=lambda x: x.replace(" ","_")).dropna()
gps_df["Device"] = gps_df["device"].str.slice(-1).astype(int)
gps_df.dropna()

pc1.parquet
pc2.parquet
pc3.parquet
pc4.parquet


,ts_gps,Latitude,Longitude,Altitude,speed_kmh,COG,precipIntensity,precipProbability,temperature,apparentTemperature,...,cloudCover,uvIndex,visibility,Traffic_Jam_Factor,Traffic_Street_Name,Traffic_Distance,Pos_in_Ref_Round,device,area,Device
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-06-22 09:52:20+02:00,2021-06-22 09:52:20,52.513433,13.332703,40.5,27.4096,260.7,0.0653,0.04,18.13,18.13,...,0.97,3.0,16.093,2.73675,Bachstraße,20.855937,15130.561672,pc1,Avenue,1
2021-06-22 09:52:21+02:00,2021-06-22 09:52:21,52.513422,13.332588,40.4,26.8540,263.6,0.0653,0.04,18.13,18.13,...,0.97,3.0,16.093,2.73675,Bachstraße,20.362597,15118.046494,pc1,Avenue,1
2021-06-22 09:52:22+02:00,2021-06-22 09:52:22,52.513417,13.332478,40.2,25.7428,268.2,0.0653,0.04,18.13,18.13,...,0.97,3.0,16.093,3.03573,Ernst-Reuter-Platz,4.984398,15118.046494,pc1,Avenue,1
2021-06-22 09:52:24+02:00,2021-06-22 09:52:24,52.513408,13.332295,40.0,17.5940,259.8,0.0653,0.04,18.13,18.13,...,0.97,3.0,16.093,3.03573,Ernst-Reuter-Platz,5.097381,15105.632141,pc1,Avenue,1
2021-06-22 09:52:25+02:00,2021-06-22 09:52:25,52.513398,13.332230,40.0,13.5196,257.2,0.0653,0.04,18.13,18.13,...,0.97,3.0,16.093,3.03573,Ernst-Reuter-Platz,4.326843,15093.329804,pc1,Avenue,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-24 10:20:34+02:00,2021-06-24 10:20:34,52.514095,13.351205,31.9,0.0000,36.6,0.0549,0.06,18.75,18.75,...,0.98,3.0,16.093,1.59728,Großer Stern,16.450398,16392.861139,pc4,Park,4
2021-06-24 10:20:35+02:00,2021-06-24 10:20:35,52.514095,13.351205,31.9,0.0000,36.6,0.0549,0.06,18.75,18.75,...,0.98,3.0,16.093,1.59728,Großer Stern,16.450398,16392.861139,pc4,Park,4
2021-06-24 10:20:36+02:00,2021-06-24 10:20:36,52.514095,13.351205,31.9,0.0000,36.6,0.0549,0.06,18.75,18.75,...,0.98,3.0,16.093,1.59728,Großer Stern,16.450398,16392.861139,pc4,Park,4


## Step: 3 Merge GPS data with PCAP data

In [14]:
df_merged = pd.merge(df_temp, gps_df.rename(columns=lambda x: x+"_source"), left_on=['timestamp', 'Source'], right_on=['timestamp', 'Device_source'])
df_merged = pd.merge(df_merged, gps_df.rename(columns=lambda x: x+"_destination"), left_on=['timestamp', 'Destination'], right_on=['timestamp', 'Device_destination'])

In [15]:
syncref_id = 3
df_merged = df_merged.assign(syncref_source=(df_merged["Source"] == syncref_id), syncref_destination=(df_merged["Destination"] == syncref_id))

In [16]:
ref_lat, ref_long = gps_df[["Latitude", "Longitude"]].mean() 
src_x, src_y = spherical_coords_to_planar_coords(df_merged.Latitude_source, df_merged.Longitude_source, ref_lat, ref_long)
dst_x, dst_y = spherical_coords_to_planar_coords(df_merged.Latitude_destination, df_merged.Longitude_destination, ref_lat, ref_long)
df_merged["distance"] = np.sqrt((src_x-dst_x)**2 + (src_y-dst_y)**2)

In [17]:
df_merged.columns

Index(['Source', 'Destination', 'Scenario', 'time_epoch', 'SNR', 'RSRP',
       'RSSI', 'NOISE POWER', 'TIME(S)', 'TIME(FrS)', 'RX_GAIN',
       'SubFrame_NUMBER', 'SubFrame_LENGHT', 'Rx_power', 'MCS',
       'Received Packets', 'ts_gps_source', 'Latitude_source',
       'Longitude_source', 'Altitude_source', 'speed_kmh_source', 'COG_source',
       'precipIntensity_source', 'precipProbability_source',
       'temperature_source', 'apparentTemperature_source', 'dewPoint_source',
       'humidity_source', 'pressure_source', 'windSpeed_source',
       'cloudCover_source', 'uvIndex_source', 'visibility_source',
       'Traffic_Jam_Factor_source', 'Traffic_Street_Name_source',
       'Traffic_Distance_source', 'Pos_in_Ref_Round_source', 'device_source',
       'area_source', 'Device_source', 'ts_gps_destination',
       'Latitude_destination', 'Longitude_destination', 'Altitude_destination',
       'speed_kmh_destination', 'COG_destination',
       'precipIntensity_destination', 'precipPro

In [18]:
# drop unnecessary column
df_merged = df_merged.drop(['TIME(S)', 'TIME(FrS)', 'Device_source', 'Device_destination'], axis=1)
df_merged

,Source,Destination,Scenario,time_epoch,SNR,RSRP,RSSI,NOISE POWER,RX_GAIN,SubFrame_NUMBER,...,visibility_destination,Traffic_Jam_Factor_destination,Traffic_Street_Name_destination,Traffic_Distance_destination,Pos_in_Ref_Round_destination,device_destination,area_destination,syncref_source,syncref_destination,distance
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-06-22 09:51:48+02:00,2,4,S1,1.624348e+09,14.144092,-71.160341,-41.737488,0.002013,44.0,426.941176,...,16.093,2.73675,Bachstraße,16.470066,15118.046494,pc4,Avenue,False,False,30.631466
2021-06-22 09:51:48+02:00,4,2,S1,1.624348e+09,15.227671,-68.752400,-46.970770,0.004633,45.0,463.300000,...,16.093,3.08926,Ernst-Reuter-Platz,4.743369,15156.001352,pc2,Avenue,False,False,30.631466
2021-06-22 09:51:49+02:00,4,2,S1,1.624348e+09,15.273688,-67.572225,-45.746560,0.006282,45.0,102.500000,...,16.093,2.73675,Bachstraße,17.991729,15143.190884,pc2,Avenue,False,False,27.982724
2021-06-22 09:51:49+02:00,2,4,S1,1.624348e+09,14.295098,-70.777137,-46.170142,0.002737,44.0,100.526316,...,16.093,3.03573,Ernst-Reuter-Platz,0.015063,15118.046494,pc4,Avenue,False,False,27.982724
2021-06-22 09:51:50+02:00,4,2,S1,1.624348e+09,15.494669,-69.356312,-47.530865,0.007427,45.0,200.294118,...,16.093,2.73675,Bachstraße,17.263233,15130.561672,pc2,Avenue,False,False,24.855905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 17:07:00+02:00,2,4,S2,1.624461e+09,12.810686,-77.330367,-49.631543,0.000524,40.0,493.761905,...,16.093,0.84534,Invalidenstraße,24.511920,380.041232,pc4,Park,False,False,13.994021
2021-06-23 17:07:00+02:00,2,3,S2,1.624461e+09,7.966860,-74.786447,-47.084520,0.002939,46.0,491.200000,...,16.093,0.84534,Invalidenstraße,5.124826,359.306424,pc3,Park,False,True,16.196722
2021-06-23 17:07:00+02:00,4,2,S2,1.624461e+09,13.532171,-78.470617,-50.735069,0.000349,40.0,504.914286,...,16.093,0.84534,Invalidenstraße,10.529770,369.711773,pc2,Park,False,False,13.994021


In [19]:
df_merged['Scenario'].unique()

array(['S1', 'S2'], dtype=object)

In [20]:
scen_info = pd.read_csv(info_path/"sidelink_scenarios.csv", index_col='scenario')
scen_info

,Packet_transmission_rate_hz,Sub_channels
scenario,,
S1,20,2
S2,50,10


In [21]:
df_merged['Packet_transmission_rate_hz'] = df_merged["Scenario"].map(scen_info['Packet_transmission_rate_hz'])
df_merged['Sub_channels'] = df_merged["Scenario"].map(scen_info['Sub_channels'])

df_merged

,Source,Destination,Scenario,time_epoch,SNR,RSRP,RSSI,NOISE POWER,RX_GAIN,SubFrame_NUMBER,...,Traffic_Street_Name_destination,Traffic_Distance_destination,Pos_in_Ref_Round_destination,device_destination,area_destination,syncref_source,syncref_destination,distance,Packet_transmission_rate_hz,Sub_channels
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-06-22 09:51:48+02:00,2,4,S1,1.624348e+09,14.144092,-71.160341,-41.737488,0.002013,44.0,426.941176,...,Bachstraße,16.470066,15118.046494,pc4,Avenue,False,False,30.631466,20,2
2021-06-22 09:51:48+02:00,4,2,S1,1.624348e+09,15.227671,-68.752400,-46.970770,0.004633,45.0,463.300000,...,Ernst-Reuter-Platz,4.743369,15156.001352,pc2,Avenue,False,False,30.631466,20,2
2021-06-22 09:51:49+02:00,4,2,S1,1.624348e+09,15.273688,-67.572225,-45.746560,0.006282,45.0,102.500000,...,Bachstraße,17.991729,15143.190884,pc2,Avenue,False,False,27.982724,20,2
2021-06-22 09:51:49+02:00,2,4,S1,1.624348e+09,14.295098,-70.777137,-46.170142,0.002737,44.0,100.526316,...,Ernst-Reuter-Platz,0.015063,15118.046494,pc4,Avenue,False,False,27.982724,20,2
2021-06-22 09:51:50+02:00,4,2,S1,1.624348e+09,15.494669,-69.356312,-47.530865,0.007427,45.0,200.294118,...,Bachstraße,17.263233,15130.561672,pc2,Avenue,False,False,24.855905,20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 17:07:00+02:00,2,4,S2,1.624461e+09,12.810686,-77.330367,-49.631543,0.000524,40.0,493.761905,...,Invalidenstraße,24.511920,380.041232,pc4,Park,False,False,13.994021,50,10
2021-06-23 17:07:00+02:00,2,3,S2,1.624461e+09,7.966860,-74.786447,-47.084520,0.002939,46.0,491.200000,...,Invalidenstraße,5.124826,359.306424,pc3,Park,False,True,16.196722,50,10
2021-06-23 17:07:00+02:00,4,2,S2,1.624461e+09,13.532171,-78.470617,-50.735069,0.000349,40.0,504.914286,...,Invalidenstraße,10.529770,369.711773,pc2,Park,False,False,13.994021,50,10


In [22]:
achieved_rate = df_merged['Received Packets']/period_seconds
packet_error = (df_merged['Packet_transmission_rate_hz'] - achieved_rate)/df_merged['Packet_transmission_rate_hz']
df_merged=df_merged.assign(Packet_error_ratio=packet_error)
df_merged

,Source,Destination,Scenario,time_epoch,SNR,RSRP,RSSI,NOISE POWER,RX_GAIN,SubFrame_NUMBER,...,Traffic_Distance_destination,Pos_in_Ref_Round_destination,device_destination,area_destination,syncref_source,syncref_destination,distance,Packet_transmission_rate_hz,Sub_channels,Packet_error_ratio
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-06-22 09:51:48+02:00,2,4,S1,1.624348e+09,14.144092,-71.160341,-41.737488,0.002013,44.0,426.941176,...,16.470066,15118.046494,pc4,Avenue,False,False,30.631466,20,2,0.15
2021-06-22 09:51:48+02:00,4,2,S1,1.624348e+09,15.227671,-68.752400,-46.970770,0.004633,45.0,463.300000,...,4.743369,15156.001352,pc2,Avenue,False,False,30.631466,20,2,0.00
2021-06-22 09:51:49+02:00,4,2,S1,1.624348e+09,15.273688,-67.572225,-45.746560,0.006282,45.0,102.500000,...,17.991729,15143.190884,pc2,Avenue,False,False,27.982724,20,2,0.00
2021-06-22 09:51:49+02:00,2,4,S1,1.624348e+09,14.295098,-70.777137,-46.170142,0.002737,44.0,100.526316,...,0.015063,15118.046494,pc4,Avenue,False,False,27.982724,20,2,0.05
2021-06-22 09:51:50+02:00,4,2,S1,1.624348e+09,15.494669,-69.356312,-47.530865,0.007427,45.0,200.294118,...,17.263233,15130.561672,pc2,Avenue,False,False,24.855905,20,2,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 17:07:00+02:00,2,4,S2,1.624461e+09,12.810686,-77.330367,-49.631543,0.000524,40.0,493.761905,...,24.511920,380.041232,pc4,Park,False,False,13.994021,50,10,0.58
2021-06-23 17:07:00+02:00,2,3,S2,1.624461e+09,7.966860,-74.786447,-47.084520,0.002939,46.0,491.200000,...,5.124826,359.306424,pc3,Park,False,True,16.196722,50,10,0.70
2021-06-23 17:07:00+02:00,4,2,S2,1.624461e+09,13.532171,-78.470617,-50.735069,0.000349,40.0,504.914286,...,10.529770,369.711773,pc2,Park,False,False,13.994021,50,10,0.30


In [23]:
write_meta(df_merged, info_path/"sidelink_info.csv")
df_merged.to_parquet(data_path/'sidelink_dataframe.parquet', compression='gzip')